In [ ]:
%%capture
# { display-mode: 'form' }

# @title PyTTI-Tools [EzMode]: VQGAN

# @markdown ## Setup

# @markdown This may take a few minutes. 

## 1. Install stuff

try: 
    import pytti
except ImportError:
    !pip install kornia pytorch-lightning transformers
    !pip install jupyter loguru einops PyGLM ftfy regex tqdm hydra-core exrex
    !pip install seaborn adjustText bunch matplotlib-label-lines
    !pip install --upgrade gdown

    !pip install --upgrade git+https://github.com/pytti-tools/AdaBins.git
    !pip install --upgrade git+https://github.com/pytti-tools/GMA.git
    !pip install --upgrade git+https://github.com/pytti-tools/taming-transformers.git
    !pip install --upgrade git+https://github.com/openai/CLIP.git
    !pip install --upgrade git+https://github.com/pytti-tools/pytti-core.git

    # These are notebook specific
    !pip install --upgrade natsort

try:
    import mmc
except:
    # install mmc
    !git clone https://github.com/dmarx/Multi-Modal-Comparators
    !pip install poetry
    !cd Multi-Modal-Comparators; poetry build
    !cd Multi-Modal-Comparators; pip install dist/mmc*.whl
    !python Multi-Modal-Comparators/src/mmc/napm_installs/__init__.py

from natsort import natsorted
from omegaconf import OmegaConf
from pathlib import Path

import mmc.loaders
!python -m pytti.warmup

notebook_params = {}

def get_output_paths():
    outv = [str(p.resolve()) for p in Path('outputs/').glob('**/*.png')]
    #outv.sort()
    outv = natsorted(outv)
    return outv

In [ ]:
resume = True # @param {type:"boolean"}

if resume:
    inits = get_output_paths()
    if inits:
       notebook_params.update({
           'init_image':inits[-1],
           })

In [ ]:
# @markdown ## Basic Settings

prompts = "a photograph of albert einstein" # @param {type:"string"}
height = 512 # @param {type:"integer"}
width = 512 # @param {type:"integer"}

cell_params = {
    "scenes": prompts,
    "height":height,
    "width":width,
}

notebook_params.update(cell_params)

In [ ]:
# @markdown ## Advanced Settings

vqgan_model = "coco" # @param ["coco","sflickr","imagenet","wikiart","openimages"]

cell_params = {
    "vqgan_model": vqgan_model,
}

notebook_params.update(cell_params)

In [ ]:
invariants = """# @package _global_

## Invariant settings  ##

steps_per_frame: 50
steps_per_scene: 500

pixel_size: 1

image_model: VQGAN

use_mmc: true
mmc_models:
- architecture: clip
  publisher: openai
  id: ViT-B/16
"""

In [ ]:
from omegaconf import OmegaConf
from pathlib import Path

cfg_invariants = OmegaConf.create(invariants)
nb_cfg = OmegaConf.create(notebook_params)
conf = OmegaConf.merge(cfg_invariants, nb_cfg)

with open("config/conf/this_run.yaml", "w") as f:
    outstr = "# @package _global_\n"
    outstr += OmegaConf.to_yaml(conf)
    print(outstr)
    f.write(
        outstr
    )


#Path("config/conf/ezmode/").mkdir(parents=True, exist_ok=True)

In [ ]:
## Do the run
! python -m pytti.workhorse conf=this_run

In [ ]:
# @title Show Outputs
from IPython.display import Image, display

outputs = list(Path('outputs/').glob('**/*.png'))
outputs.sort()
im_path = str(outputs[-1])

Image(im_path, height=height, width=width)

In [ ]:
# @markdown compile images into a video of the generative process

from PIL import Image as pilImage
from subprocess import Popen, PIPE
from tqdm.notebook import tqdm

fps = 12 # @param {type:'number'}

fpaths = get_output_paths()

frames = []

for filename in tqdm(fpaths):
    frames.append(pilImage.open(filename))

cmd_in = ['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-']
cmd_out = ['-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '1', '-preset', 'veryslow', f'output.mp4']

cmd = cmd_in + cmd_out

p = Popen(cmd, stdin=PIPE)
for im in tqdm(frames):
  im.save(p.stdin, 'PNG')
p.stdin.close()

print("Encoding video...")
p.wait()
print("Video saved to output.mp4.")